# Artist| Writers| Directors

In [10]:
import pandas as pd
import sqlite3
import os
import zipfile

# Check if 'im.db' exists, unzip if it doesn't
if not os.path.exists('zippedData/im.db'):
    with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
        zip_ref.extractall('zippedData')

# Connect to the SQLite database
conn = sqlite3.connect('zippedData/im.db')

In [11]:
cursor = conn.cursor()

In [12]:
pd.read_sql("""
  SELECT *
  FROM sqlite_master  
  """, conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [14]:
pd.read_sql("""
SELECT mb.genres, mr.averagerating AS avg_ratings, mr.numvotes AS num_votes,p.primary_name AS artist_name, mb.primary_title AS movie_title
FROM movie_basics mb
JOIN movie_ratings mr ON mr.movie_id = mb.movie_id
JOIN principals pr ON pr.movie_id = mb.movie_id
JOIN persons p ON p.person_id = pr.person_id
WHERE p.death_year IS NULL
;""", conn)

,genres,avg_ratings,num_votes,artist_name,movie_title
0,"Adventure,Animation,Comedy",6.0,15612,Mary Ellen Bauder,Smurfs: The Lost Village
1,"Comedy,Drama,Music",4.7,214,Joseph Bauer,Satin
2,"Comedy,Horror",4.0,127,Joseph Bauer,Caesar and Otto's Deadly Xmas
3,"Comedy,Family,Fantasy",3.6,582,Joseph Bauer,Elf-Man
4,"Action,Adventure,Family",5.4,50,Joseph Bauer,Journey to the Forbidden Valley
...,...,...,...,...,...
621361,"Comedy,Drama,Thriller",7.6,489,Szilvia Herr,Remélem legközelebb sikerül meghalnod:)
621362,Horror,3.8,341,Mariya Shekaturova,Diggery
621363,Drama,8.2,6,Susan Grobes,Choices
621364,Horror,4.6,38,Joo Yeon So,Sodak Sodak


In [35]:
pd.read_sql("""
    SELECT 
        w.*, 
        p.primary_name, 
        ma.title,
        mb.genres,
        ma.language,
        mr.averagerating,
        mr.numvotes
    FROM writers w
    JOIN persons p ON w.person_id = p.person_id
    JOIN movie_basics mb  ON w.movie_id = mb.movie_id
    JOIN movie_akas ma ON w.movie_id = ma.movie_id
    JOIN movie_ratings mr ON w.movie_id = mr.movie_id
    JOIN principals pr USING(movie_id)
    WHERE p.death_year IS NULL
    AND p.primary_profession = 'writer'
    AND (mb.genres LIKE '%Horror%' OR mb.genres LIKE '%Thriller%' OR mb.genres LIKE'%Mystery%')
    AND mr.averagerating >= 7.5
    AND mr.numvotes > 9665
    GROUP BY primary_name
    ORDER BY mr.averagerating DESC
""", conn)

,movie_id,person_id,primary_name,title,genres,language,averagerating,numvotes
0,tt5354160,nm7861818,Adnan Adib Khan,Aynabaji,"Crime,Mystery,Thriller",None,9.3,18470
1,tt8108198,nm4914249,Arijit Biswas,Andhadhun,"Crime,Thriller",None,8.5,43409
2,tt4987556,nm5394336,A.N. Balakrishnan,Double Attack 2,"Action,Crime,Thriller",hi,8.5,13747
3,tt8239946,nm10304029,Narayan Dharap,Tumbbad,"Drama,Fantasy,Horror",None,8.3,14128
4,tt2375559,nm6253319,Hari Prasad Jakka,1 - Nenokkadine,"Action,Mystery,Thriller",None,8.3,41124
5,tt3461252,nm6211716,Ahmed Mourad,Al Feel Al Azrak,"Drama,Horror,Mystery",ar,8.2,21776
6,tt4016934,nm1941029,Seo-kyeong Jeong,A Criada,"Drama,Romance,Thriller",None,8.1,79272
7,tt4016934,nm1258797,Sarah Waters,A Criada,"Drama,Romance,Thriller",None,8.1,79272
8,tt1821480,nm4449370,Nikhil Vyas,A sztori,"Mystery,Thriller",None,8.1,51830
9,tt1821480,nm4298729,Advaita Kala,A sztori,"Mystery,Thriller",None,8.1,51830


In [36]:
pd.read_sql("""
SELECT *
FROM movie_basics mb
JOIN directors dr USING(movie_id)
JOIN principals pr USING(movie_id)
JOIN persons p USING(person_id)
JOIN movie_ratings mr USING(movie_id)
WHERE death_year IS NULL
AND
(
primary_title LIKE '%Gallows%' 
OR primary_title LIKE '%Devil Inside%' 
OR primary_title = '%Insidious%' 
OR primary_title = '%Unfriended%' 
OR primary_title LIKE '%Paranormal Activity%' 
OR primary_title = '%Split%' 
OR primary_title = '%Chernobyl Diaries%' 
OR primary_title = '%Annabelle%'
)
AND 
category = 'director'
GROUP BY primary_name
;
""", conn)

,movie_id,primary_title,start_year,runtime_minutes,genres,person_id,ordering,person_id,category,job,characters,primary_name,birth_year,death_year,primary_profession,averagerating,numvotes
0,tt3108988,Paranormal Activity Security Squad,2016,NaN,"Comedy,Horror",nm2066764,5,nm2066764,director,None,None,Alexander Wraith,1979.0,None,"actor,writer,producer",5.4,13
1,tt2109184,Paranormal Activity 4,2012,88.0,"Horror,Mystery,Thriller",nm1413364,5,nm1160962,director,None,None,Ariel Schulman,NaN,None,"writer,producer,director",4.6,58408
2,tt2309260,The Gallows,2015,81.0,"Horror,Mystery,Thriller",nm3951039,5,nm4000389,director,None,None,Chris Lofing,NaN,None,"producer,director,writer",4.2,17763
3,tt2473682,Paranormal Activity: The Marked Ones,2014,84.0,"Horror,Mystery,Thriller",nm0484907,5,nm0484907,director,None,None,Christopher Landon,1975.0,None,"writer,producer,director",5.0,37147
4,tt2417650,30 Nights of Paranormal Activity with the Devi...,2013,80.0,Comedy,nm0608936,5,nm0608936,director,None,None,Craig Moss,NaN,None,"director,writer,producer",2.6,3381
5,tt2473510,Paranormal Activity: The Ghost Dimension,2015,88.0,"Horror,Mystery,Thriller",nm0687427,5,nm0687427,director,None,None,Gregory Plotkin,NaN,None,"editorial_department,editor,producer",4.6,22361
6,tt2109184,Paranormal Activity 4,2012,88.0,"Horror,Mystery,Thriller",nm1160962,5,nm1160962,director,None,None,Henry Joost,NaN,None,"writer,producer,director",4.6,58408
7,tt1990211,The Devil Inside Me,2011,90.0,"Horror,Thriller",nm3814942,5,nm3814942,director,None,None,Qi Zhang,NaN,None,"writer,director,actor",5.6,59
8,tt1649749,Demon Exorcism: The Devil Inside Maxwell Bastas,2013,NaN,Horror,nm3157118,5,nm3157118,director,None,None,Richard G. James,NaN,None,"director,writer,editor",5.5,35
9,tt1536044,Paranormal Activity 2,2010,91.0,Horror,nm0931095,5,nm0931095,director,None,None,Tod Williams,1968.0,None,"director,writer,producer",5.7,93122


In [43]:
pd.read_sql("""
SELECT *
FROM movie_basics mb
JOIN directors dr USING(movie_id)
JOIN principals pr USING(movie_id)
JOIN persons p USING(person_id)
JOIN movie_ratings mr USING(movie_id)
WHERE death_year IS NULL
AND (
    primary_title LIKE '%Gallows%' 
    OR primary_title LIKE '%Devil Inside%' 
    OR primary_title LIKE '%Insidious%' 
    OR primary_title LIKE '%Unfriended%' 
    OR primary_title LIKE '%Paranormal Activity%' 
    OR primary_title LIKE '%Split%' 
    OR primary_title LIKE '%Chernobyl Diaries%' 
    OR primary_title LIKE '%Annabelle%'
)
AND mr.averagerating >= 7.5
AND category IN ('actor', 'actress')
GROUP BY primary_name;
""", conn)

,movie_id,primary_title,start_year,runtime_minutes,genres,person_id,ordering,person_id,category,job,characters,primary_name,birth_year,death_year,primary_profession,averagerating,numvotes
0,tt1537385,Annabelle & Bear,2010,100.0,Drama,nm3672532,1,nm0557478,actor,None,"[""Bear""]",Amy S. Weber,NaN,None,"producer,director,writer",8.0,53
1,tt3168478,Split Gas,2014,71.0,"Comedy,Drama",nm4839546,1,nm4587628,actress,None,"[""Otey""]",Jacques Edeline,NaN,None,"writer,actor,sound_department",8.3,7
2,tt8451712,Babysplitters,2019,119.0,Comedy,nm1153627,1,nm0275417,actor,None,"[""Dr. Cooper, MFT""]",Sam Friedlander,1978.0,None,"producer,production_manager,director",9.3,77
3,tt2467370,Split Second,2012,83.0,Drama,nm5394959,1,nm4244738,actor,None,"[""Mike Yancey""]",Timothy Fontaine,NaN,None,"cinematographer,editor,director",8.1,11
